# 라이브러리

In [1]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
from pathlib import Path
import pandas as pd
import cv2
import os
from ultralytics import YOLO

# YOLO모델 load 및 파일 경로

In [11]:
# 로컬에 저장된 YOLO 모델의 가중치 파일 경로 설정 C:\Users\User\Desktop
model_weights = 'C:/Users/User/Desktop/Auto Labeling/best.pt'  # 실제 경로로 변경

# # YOLOv5 모델 로드
# model = torch.hub.load('ultralytics/yolov8n', 'custom', path=model_weights)

# YOLOv8n
model = YOLO(model_weights)

# 라벨링할 데이터 경로 설정
dataset_directory = 'C:/Users/User/Desktop/Auto Labeling/contrast_up/images'
output_directory = 'C:/Users/User/Desktop/Auto Labeling/contrast_up/labels'
Path(output_directory).mkdir(parents=True, exist_ok=True)

# CSV -> YOLO 라벨형식 변환

In [6]:
# YOLO 포맷의 라벨링 파일 저장 함수
def save_yolo_format(label_data, label_file_path, img_height, img_width):

    with open(label_file_path, 'a') as f:  # 'a' 모드를 사용하여 파일에 내용을 추가합니다.
        for _, row in label_data.iterrows():
            class_id = int(row['class'])

            # 바운딩 박스 좌표 계산
            x_center = ((row['xmin'] + row['xmax']) / 2) / img_width
            y_center = ((row['ymin'] + row['ymax']) / 2) / img_height
            width = (row['xmax'] - row['xmin']) / img_width
            height = (row['ymax'] - row['ymin']) / img_height
            # YOLO 포맷으로 라벨 기록
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

# 오토라벨링

label만

In [7]:
# 오토라벨링 함수 정의
def autolabel_images(directory):
    current_img_name = None
    label_file_path = None

    for img_path in Path(directory).glob('*.png'):  # .png 확장자의 이미지 파일 처리
        results = model(str(img_path))  # 이미지 예측 수행
        result = results[0]  # 결과의 첫 번째 요소 가져오기
        boxes = result.boxes  # 바운딩 박스 정보 가져오기
        
        # 이미지 크기 가져오기
        img = cv2.imread(str(img_path))
        img_height, img_width = img.shape[:2]  # 이미지의 height와 width 추출

        # boxes 객체를 데이터프레임으로 변환
        if boxes is not None and len(boxes) > 0:
            labels = boxes.data.cpu().numpy()  # 바운딩 박스 데이터를 NumPy 배열로 변환
            labels_df = pd.DataFrame(labels, columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class'])

            # 이미지 파일 이름이 바뀌면 새로 설정
            if img_path.stem != current_img_name:
                current_img_name = img_path.stem
                label_file_path = Path(output_directory) / f"{current_img_name}.txt"
                # 기존의 동일한 이름의 파일이 있으면 삭제
                if label_file_path.exists():
                    label_file_path.unlink()

            if not labels_df.empty:
                save_yolo_format(labels_df, label_file_path, img_height, img_width)
            else:
                print(f"No objects detected in {img_path.name}")

# 실행

In [12]:
# 오토라벨링 실행
autolabel_images(dataset_directory)


image 1/1 C:\Users\User\Desktop\Auto Labeling\contrast_up\images\202301060737399559064817.png: 640x480 1 lcd_tv, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\User\Desktop\Auto Labeling\contrast_up\images\202301110550574739059044.png: 480x640 1 vending_machine_s, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 C:\Users\User\Desktop\Auto Labeling\contrast_up\images\202301110629385809083216.png: 640x480 (no detections), 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 C:\Users\User\Desktop\Auto Labeling\contrast_up\images\202301110645107369076908.png: 640x288 1 dishwasher, 9.0ms
Speed: 1.0ms preprocess, 9.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 288)
